In [1]:
import pandas as pd
import datetime as dt

USvideos_data = pd.read_csv('Resources/USvideos.zip')
CAvideos_data = pd.read_csv('Resources/CAvideos.zip')

USvideos_data['country'] = 'USA'
CAvideos_data['country'] = 'Canada'

In [2]:
USvideos_data['trending_date'] = '20'+ USvideos_data['trending_date']
USvideos_data['trending_date'] = USvideos_data['trending_date'].str.replace('.','-')
USvideos_data['trending_date'] = USvideos_data['trending_date'].apply(lambda x: dt.datetime.strptime(x,'%Y-%d-%m'))

CAvideos_data['trending_date'] = '20'+ CAvideos_data['trending_date']
CAvideos_data['trending_date'] = CAvideos_data['trending_date'].str.replace('.','-')
CAvideos_data['trending_date'] = CAvideos_data['trending_date'].apply(lambda x: dt.datetime.strptime(x,'%Y-%d-%m'))

In [3]:
# 2018 June trending data is the latest data available at this moment
date_rangeUS = (USvideos_data['trending_date'] > '2018-06-01') & (USvideos_data['trending_date'] <= '2018-06-30')
USvideos_data = USvideos_data.loc[date_rangeUS]

date_rangeCA = (CAvideos_data['trending_date'] > '2018-06-01') & (CAvideos_data['trending_date'] <= '2018-06-30')
CAvideos_data = CAvideos_data.loc[date_rangeCA]

In [4]:
# Find & Count the null columns
null_columns = USvideos_data.columns[USvideos_data.isnull().any()]
print(USvideos_data[null_columns].isnull().sum())

# Locate and inspect null values within data
USvideos_data[USvideos_data.isnull().any(axis=1)][null_columns]

description    20
dtype: int64


,description
38484,NaN
38526,NaN
38689,NaN
38729,NaN
38892,NaN
38932,NaN
39096,NaN
39136,NaN
39302,NaN
39339,NaN


In [5]:
# Complete the null data value
datatype_USVideos = USvideos_data.dtypes.to_dict()
USvideos_data['description'].fillna('None', inplace=True, downcast=datatype_USVideos)
print(USvideos_data.loc[USvideos_data['description'].isnull()])

datatype_CAVideos = USvideos_data.dtypes.to_dict()
CAvideos_data['description'].fillna('None', inplace=True, downcast=datatype_CAVideos)
print(CAvideos_data.loc[CAvideos_data['description'].isnull()])

Empty DataFrame
Columns: [video_id, trending_date, title, channel_title, category_id, publish_time, tags, views, likes, dislikes, comment_count, thumbnail_link, comments_disabled, ratings_disabled, video_error_or_removed, description, country]
Index: []
Empty DataFrame
Columns: [video_id, trending_date, title, channel_title, category_id, publish_time, tags, views, likes, dislikes, comment_count, thumbnail_link, comments_disabled, ratings_disabled, video_error_or_removed, description, country]
Index: []


In [6]:
# Export and backup cleaned data
export_USvideos = USvideos_data.to_csv('Output/USvideos_clean.csv')
export_CAvideos = CAvideos_data.to_csv('Output/CAvideos_clean.csv')

In [7]:
# Combine two dataset
combine_df = pd.merge(USvideos_data,CAvideos_data, how='outer')

In [8]:
category_idname = pd.read_csv('Resources/YT_Category_idname.csv', header=None)
category_idname.columns = ['category_id','category_name']

In [9]:
export_cat_id = category_idname.to_csv('Output/Category_id.csv')

In [10]:
combine_df = combine_df.merge(category_idname, how='left', on='category_id')

In [11]:
combine_df['publish_time'] = pd.to_datetime(combine_df['publish_time'])
combine_df['publish_time'] = combine_df['publish_time'].dt.date

In [12]:
export_df = combine_df.to_csv('Output/Combined_data.csv')

In [13]:
# Inspect duplicates for ERD ideas

find_dupe = combine_df[combine_df.duplicated(['video_id'],keep=False)]
print(find_dupe.video_id.count())
find_dupe.sort_values(['video_id']).head(4)

4117


,video_id,trending_date,title,channel_title,category_id,publish_time,tags,views,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,video_error_or_removed,description,country,category_name
1026,-0CMnp02rNY,2018-06-07,Mindy Kaling's Daughter Had the Perfect Reacti...,TheEllenShow,24,2018-06-04,"ellen|""ellen degeneres""|""the ellen show""|""elle...",605506,7848,232,354,https://i.ytimg.com/vi/-0CMnp02rNY/default.jpg,False,False,False,Ocean's 8 star Mindy Kaling dished on bringing...,USA,Entertainment
1658,-0CMnp02rNY,2018-06-10,Mindy Kaling's Daughter Had the Perfect Reacti...,TheEllenShow,24,2018-06-04,"ellen|""ellen degeneres""|""the ellen show""|""elle...",792613,9720,330,413,https://i.ytimg.com/vi/-0CMnp02rNY/default.jpg,False,False,False,Ocean's 8 star Mindy Kaling dished on bringing...,USA,Entertainment
820,-0CMnp02rNY,2018-06-06,Mindy Kaling's Daughter Had the Perfect Reacti...,TheEllenShow,24,2018-06-04,"ellen|""ellen degeneres""|""the ellen show""|""elle...",475965,6531,172,271,https://i.ytimg.com/vi/-0CMnp02rNY/default.jpg,False,False,False,Ocean's 8 star Mindy Kaling dished on bringing...,USA,Entertainment
1859,-0CMnp02rNY,2018-06-11,Mindy Kaling's Daughter Had the Perfect Reacti...,TheEllenShow,24,2018-06-04,"ellen|""ellen degeneres""|""the ellen show""|""elle...",800359,9773,332,423,https://i.ytimg.com/vi/-0CMnp02rNY/default.jpg,False,False,False,Ocean's 8 star Mindy Kaling dished on bringing...,USA,Entertainment
